# TABELLA PATTERN SELEZIONATI DA EURISTICA/RARI/COMUNI

In [1]:
"""
Script per:
1) caricare topk_riconteggio.csv in un PatternTable
2) selezionare pattern con l'euristica (o rari/comuni)
3) aggiungere la colonna Velocity
4) esportare in Excel i pattern selezionati con:
   - frequenza (p)
   - pattern in formato lista
   - entropia
   - entropia / entropia totale
   - Dinamicity (Velocity)
"""

from pathlib import Path
import pandas as pd

from motionpattern.pattern_table import (
    ImportData,
    apply_heuristic,
    apply_rari_match_info,
    apply_comuni_match_info,
    apply_rari_match_N_W,
    apply_comuni_match_N_W,
    add_velocity,
)

# ==========================================================
# PARAMETRI PRINCIPALI DA CAMBIARE IN BASE AI DATI
# ==========================================================

ROOT = Path().resolve().parents[0]

# File CSV con le probabilità p già stimate (topk_riconteggio.csv)
csv_path = ROOT / "examples" / "data" / "topk_riconteggio.csv"

# TimeFilter e SpaceFilter del dataset (es. 3×3×3)
TIME_FILTER = 3    # per pattern statici sarebbe 1
SPACE_FILTER = 3

# Parametri dell'euristica
MAX_N = 300        # numero massimo di pattern
MAX_W = 0.025     # banda massima consentita

# File Excel di output
xlsx_out = ROOT / "output" / f"pattern_selezionati_N_W_{MAX_N}_{MAX_W}.xlsx"

# ==========================================================
# 1) Importa il CSV in un PatternTable
# ==========================================================

table = ImportData(
    path=csv_path,
    real_counting=True,     # usa la colonna 'p' del file
    TimeFilter=TIME_FILTER,
    SpaceFilter=SPACE_FILTER,
)

print("Tabella caricata.")
table.print_info(masked=True)

# ==========================================================
# 2) Selezione pattern
#
# Di default: euristica su N e W.
# Se si vogliono pattern rari/comuni, si possono sostituire le righe
# con una delle funzioni sotto.
# ==========================================================

# --- euristica standard ---
table_sel = apply_heuristic(
    df=table,
    Max_W=MAX_W,
    Max_N=MAX_N,
    inplace=False,   # crea una copia, non modifica la tabella originale
)

# --- esempi alternativi (da usare al posto di apply_heuristic) ---
# pattern rari con match sull'informazione:
# table_sel, skipped = apply_rari_match_info(df=table, Max_W=MAX_W, Max_N=MAX_N, inplace=False)
#
# pattern comuni con match sull'informazione:
# table_sel, skipped = apply_comuni_match_info(df=table, Max_W=MAX_W, Max_N=MAX_N, inplace=False)
#
# pattern rari con vincoli diretti N, W:
# table_sel, skipped = apply_rari_match_N_W(df=table, Max_W=MAX_W, Max_N=MAX_N, inplace=False)
#
# pattern comuni con vincoli diretti N, W:
# table_sel, skipped = apply_comuni_match_N_W(df=table, Max_W=MAX_W, Max_N=MAX_N, inplace=False)

print("\nDopo selezione:")
table_sel.print_info(masked=True)

# ==========================================================
# 3) Aggiungi la colonna Velocity (definita solo se TimeFilter > 1)
# ==========================================================

add_velocity(table_sel, inplace=True)

# ==========================================================
# 4) Costruisci la tabella dei soli pattern selezionati e salvala in Excel
# ==========================================================

# Sottoinsieme dei pattern selezionati (Mask=True)
df_sel = table_sel[table_sel["Mask"]].copy()

# Pattern in formato lista (TimeFilter × SpaceFilter × SpaceFilter
# oppure SpaceFilter × SpaceFilter se TimeFilter == 1)
pattern_arrays = table_sel.get_PatternList(masked=True)
df_sel["Pattern_list"] = [arr.tolist() for arr in pattern_arrays]

# Seleziona e rinomina le colonne per l’Excel finale
df_excel = df_sel[["p", "Pattern_list", "Entropy", "EntropyRatio", "Velocity"]].copy()
df_excel = df_excel.rename(
    columns={
        "p": "Frequency",
        "Pattern_list": "Pattern_list",
        "Entropy": "Entropy",
        "EntropyRatio": "EntropyRatio",
        "Velocity": "Dinamicity",   
    }
)

# Crea cartella di output se non esiste
xlsx_out.parent.mkdir(parents=True, exist_ok=True)

# Salva in Excel, usando l'indice come PatternValue
df_excel.to_excel(xlsx_out, index_label="PatternValue")

print(f"\nFile Excel salvato in: {xlsx_out}")
print(f"Numero di pattern selezionati: {len(df_excel)}")


Tabella caricata.
Numero di pattern selezionati: 121890
Larghezza di banda (somma p): 0.915718
Entropia totale: 4.030601
Somma EntropyRatio: 1.000000

Dopo selezione:
Numero di pattern selezionati: 236
Larghezza di banda (somma p): 0.024951
Entropia totale: 0.228103
Somma EntropyRatio: 0.056593

File Excel salvato in: C:\Users\braua\Documents\TesiMagistrale\repository\Pattern-Motion-Project\output\pattern_selezionati_N_W_300_0.025.xlsx
Numero di pattern selezionati: 236
